## Implementation

In [13]:
from IOHMM import IOHMM_model
import numpy as np, pandas as pd
import torch

In [14]:
data = pd.read_csv("data/Financial-Data/stocks/TSLA.csv")
data.head()
data = data.dropna()
input_data = data[['Open', 'High', 'Low', 'Close']][:-1]


# close price of the next day
output_data = data['Close'][1:]
# to start from 0 index
output_data.index = range(len(output_data))

output_data.name = 'Close-next-day'

data = pd.concat([input_data, output_data], axis=1)

data.head()
print(len(data))


input = torch.tensor(np.array(data[['Open', 'High', 'Low', 'Close']]), dtype=torch.float32)
output = torch.tensor(np.array(data['Close-next-day']), dtype=torch.float32)

print(output[0:5])
print(input[0:5])
torch.cat((torch.tensor([1.0]), output))


3259
tensor([1.5887, 1.4640, 1.2800, 1.0740, 1.0533])
tensor([[1.2667, 1.6667, 1.1693, 1.5927],
        [1.7193, 2.0280, 1.5533, 1.5887],
        [1.6667, 1.7280, 1.3513, 1.4640],
        [1.5333, 1.5400, 1.2473, 1.2800],
        [1.3333, 1.3333, 1.0553, 1.0740]])


tensor([  1.0000,   1.5887,   1.4640,  ..., 224.5700, 234.8600, 244.4000])

In [15]:
data = pd.read_csv("data/Financial-Data/stocks/TSLA.csv")
data.head()
data = data.dropna()
input_data = data[['Open', 'High', 'Low', 'Close']][:-1]


# close price of the next day
output_data = data['Close'][1:]
# to start from 0 index
output_data.index = range(len(output_data))

output_data.name = 'Close-next-day'

data = pd.concat([input_data, output_data], axis=1)

data.head()
print(len(data))


input = torch.tensor(np.array(data[['Open']]), dtype=torch.float32)
output = torch.tensor(np.array(data['Close']), dtype=torch.float32)

print(output[0:5])
print(input[0:5])
torch.cat((torch.tensor([1.0]), output))


3259
tensor([1.5927, 1.5887, 1.4640, 1.2800, 1.0740])
tensor([[1.2667],
        [1.7193],
        [1.6667],
        [1.5333],
        [1.3333]])


tensor([  1.0000,   1.5927,   1.5887,  ..., 221.3100, 224.5700, 234.8600])

# Example 1

## Set up a simple model manully

In [16]:
IOHMM = IOHMM_model(num_states=2, inputs=input[:10], outputs=output[:10], max_iter=1000, tol=1e-6)
print(IOHMM.initial_pi)
print(IOHMM.transition_matrix)
print(IOHMM.emission_matrix)
print(IOHMM.sd)

Parameter containing:
tensor([0.5000, 0.5000], requires_grad=True)
Parameter containing:
tensor([[[-1.0868, -1.4740],
         [ 0.6743, -0.3188]],

        [[-0.0385,  1.6550],
         [-2.0065, -1.5370]]], requires_grad=True)
Parameter containing:
tensor([[-1.8259, -0.7665],
        [ 2.1233,  1.1693]], requires_grad=True)
Parameter containing:
tensor([1., 1.], requires_grad=True)


In [17]:
print(IOHMM._forward())
torch.sum(IOHMM._forward(), dim=1)

tensor([[4.9527e-04, 9.9950e-01],
        [5.6645e-06, 1.5087e-02],
        [1.2045e-05, 1.3002e-02],
        [4.2781e-05, 1.2062e-02],
        [1.8740e-04, 1.2929e-02],
        [4.1190e-04, 2.4467e-02],
        [2.8781e-04, 3.2699e-02],
        [2.2032e-04, 2.5081e-02],
        [2.2337e-04, 2.2258e-02],
        [1.8794e-04, 2.9207e-02]])


tensor([1.0000, 0.0151, 0.0130, 0.0121, 0.0131, 0.0249, 0.0330, 0.0253, 0.0225,
        0.0294])

In [18]:
print(IOHMM._backward())
torch.sum(IOHMM._backward(), dim=1)

tensor([[5.6645e-06, 1.5087e-02],
        [1.2045e-05, 1.3002e-02],
        [4.2781e-05, 1.2062e-02],
        [1.8740e-04, 1.2929e-02],
        [4.1190e-04, 2.4467e-02],
        [2.8781e-04, 3.2699e-02],
        [2.2032e-04, 2.5081e-02],
        [2.2337e-04, 2.2258e-02],
        [1.8794e-04, 2.9207e-02],
        [5.7308e-03, 9.9427e-01]])


tensor([0.0151, 0.0130, 0.0121, 0.0131, 0.0249, 0.0330, 0.0253, 0.0225, 0.0294,
        1.0000])

In [19]:
print(IOHMM._compute_gamma(IOHMM._forward(), IOHMM._backward()))
torch.sum(IOHMM._compute_gamma(IOHMM._forward(), IOHMM._backward()), dim=1)

tensor([[1.8604e-07, 1.0000e+00],
        [3.4781e-07, 1.0000e+00],
        [3.2856e-06, 1.0000e+00],
        [5.1406e-05, 9.9995e-01],
        [2.4396e-04, 9.9976e-01],
        [1.4816e-04, 9.9985e-01],
        [7.7311e-05, 9.9992e-01],
        [8.8145e-05, 9.9991e-01],
        [6.4569e-05, 9.9994e-01],
        [3.7086e-05, 9.9996e-01]])


tensor([1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000,
        1.0000])

In [20]:
print(IOHMM._compute_xi(IOHMM._forward(), IOHMM._backward()))
a = torch.sum(IOHMM._compute_xi(IOHMM._forward(), IOHMM._backward()), axis=1)
torch.sum(a, axis=1)

tensor([[[1.2976e-06, 5.0000e-01],
         [1.2976e-06, 5.0000e-01]],

        [[2.4007e-07, 5.0000e-01],
         [2.4007e-07, 5.0000e-01]],

        [[6.9820e-07, 5.0000e-01],
         [6.9820e-07, 5.0000e-01]],

        [[7.0915e-06, 4.9999e-01],
         [7.0915e-06, 4.9999e-01]],

        [[3.1925e-05, 4.9997e-01],
         [3.1925e-05, 4.9997e-01]],

        [[6.9388e-05, 4.9993e-01],
         [6.9388e-05, 4.9993e-01]],

        [[8.0533e-05, 4.9992e-01],
         [8.0533e-05, 4.9992e-01]],

        [[4.7762e-05, 4.9995e-01],
         [4.7762e-05, 4.9995e-01]],

        [[3.0510e-05, 4.9997e-01],
         [3.0510e-05, 4.9997e-01]],

        [[3.1307e-05, 4.9997e-01],
         [3.1307e-05, 4.9997e-01]]])


tensor([1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000,
        1.0000])

In [21]:
IOHMM._baum_welch()

KeyboardInterrupt: 

## See the training results

In [ ]:
print("Initial pi")
print(IOHMM.initial_pi) # to normalize
print("Transition matrix")
print(IOHMM.transition_matrix)
print("Emission matrix")
print(IOHMM.emission_matrix)
print("Sd")
print(IOHMM.sd)

Initial pi
Parameter containing:
tensor([0.5000, 0.5000], requires_grad=True)
Transition matrix
Parameter containing:
tensor([[[ 0.4753,  0.6747,  0.5153, -0.1341,  0.3780],
         [ 1.0045,  0.8150, -1.0927, -0.4047,  0.8086]],

        [[ 1.2141,  2.0483,  0.5503,  0.9388, -0.9632],
         [-1.4472, -2.1822, -0.6814,  1.0127, -0.8314]]], requires_grad=True)
Emission matrix
Parameter containing:
tensor([[ 1.5415, -0.9525,  1.5101, -0.4526,  0.8942],
        [-0.0389, -0.5956, -0.2715, -0.1510, -0.0386]], requires_grad=True)
Sd
Parameter containing:
tensor([1., 1.], requires_grad=True)


## Viterbi

In [ ]:
#call the viterbi algorithm
IOHMM.viterbi()

tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0])